# Getting Model
Short, simple script to extract models from studies, convert to tflite, and save.

There seems to be an error when running sometimes? This runs fine on the pydda_env env, but not on my personal environment.

In [1]:
# baseline imports
import numpy as np
import os
import pickle
import tensorflow.lite as tflite
import tensorflow as tf

## Extract models from studies
Note that the paths are important here; change them as need may be

In [2]:
def get_best_model_from_study(path):
    file = open(path, "rb")
    study = pickle.load(file)
    file.close()
    return study.best_trial.user_attrs['model']

def convert_model_to_tflite(model):
    converter = tflite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tflite.Optimize.DEFAULT]
    converter.experimental_new_converter=True
    converter.target_spec.supported_ops = [tflite.OpsSet.TFLITE_BUILTINS, tflite.OpsSet.SELECT_TF_OPS]
    return converter.convert()
    
def save_tflite_model(tflite_model, name, dirname):
    path = os.path.join(dirname,f'{name}.tflite')
    open(path, "wb").write(tflite_model)

def convert_and_save_as_tflite(name, model):
    tflite_model = convert_model_to_tflite(model)
    save_tflite_model(tflite_model, name, dirname='models')

Seasonal models

In [7]:
dirname = '../2022 summer argonne/jupyter/!data/pre-loaded/04all_data_16in_4out_seasonal/'
subfolders = [f'season{i}' for i in range(1,5)]
study_filename = '16in.4out.study'
study_paths = [os.path.join(dirname, subfolder, study_filename) for subfolder in subfolders]



# get the best models from each study in the list and save them in a dict w/ their names
models = [get_best_model_from_study(path) for path in study_paths]
model_names = ['winter','spring','summer','fall']
models_dict = dict(zip(model_names, models))

# save each model to a file w/ its associated name
for name, model in models_dict.items():
    convert_and_save_as_tflite(name, model)

/home/jonathan.gao/.conda/envs/pydda_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-28 14:41:01.197875: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-28 14:41:03.640878: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38396 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-07-28 14:41:45.026886: W tensorflow/python/util/util.cc:368] Sets are not currently considered seque

INFO:tensorflow:Assets written to: ram://12921fe5-98dd-4f61-bad4-57afa6ade558/assets


INFO:tensorflow:Assets written to: ram://12921fe5-98dd-4f61-bad4-57afa6ade558/assets


INFO:tensorflow:Assets written to: ram://bcccdaab-bd07-42d5-a2da-2bfb7923de43/assets


INFO:tensorflow:Assets written to: ram://bcccdaab-bd07-42d5-a2da-2bfb7923de43/assets


INFO:tensorflow:Assets written to: ram://ca0f0af0-73d7-4a5e-bd8e-ffa3070714da/assets


INFO:tensorflow:Assets written to: ram://ca0f0af0-73d7-4a5e-bd8e-ffa3070714da/assets


INFO:tensorflow:Assets written to: ram://43509b4e-2052-412b-b7b5-b39bcac5c466/assets


INFO:tensorflow:Assets written to: ram://43509b4e-2052-412b-b7b5-b39bcac5c466/assets


Non-seaonal model

In [ ]:
dirname = ''
study_filename = '48in.12out.study'
study_path = os.path.join(dirname, study_filename)
print(study_path)

nonseasonal_model = get_best_model_from_study(study_path)
convert_and_save_as_tflite('3hournonseasonal', nonseasonal_model)

## Testing predictions

In [ ]:
# baseline imports
import numpy as np
import os
import pickle
import tensorflow as tf
import tensorflow.lite as tflite

gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

    
path = 'models/spring.tflite'
interpreter = tflite.Interpreter(path)
interpreter.allocate_tensors()
print('tensors allocated----------')


def unpickle_thing(path):
    with open(path, 'rb') as handle:
        thing = pickle.load(handle)
    return thing

X_test = unpickle_thing('dummy_data/X_test')
y_test = unpickle_thing('dummy_data/y_test')

X_test = np.float32(X_test)
print('data loaded----------')


output = interpreter.get_output_details()[0]  # Model has single output.
input_ = interpreter.get_input_details()[0]  # Model has single input.
input_data = np.expand_dims(X_test[0],axis=0)
interpreter.set_tensor(input_['index'], input_data)
print('about to invoke----------')
interpreter.invoke()

print(interpreter.get_tensor(output['index']))

interpreter.close()

INFO: Created TensorFlow Lite delegate for select TF ops.
2022-07-29 10:23:21.537854: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-29 10:23:25.888608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38396 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:b7:00.0, compute capability: 8.0
INFO: TfLiteFlexDelegate delegate: 6 nodes delegated out of 22 nodes with 4 partitions.



tensors allocated----------
data loaded----------
about to invoke----------
